In [1]:
import pandas as pd
from contextlib import chdir
import os
import pathlib

In [2]:
b3lyp = pd.read_csv('b3lyp_eprec6.csv')
hf = pd.read_csv('hf_eprec6.csv')
lda = pd.read_csv('lda_eprec6.csv')
pbe0 = pd.read_csv('pbe0_eprec6.csv')

In [3]:
# column of names is named Unnamed for some reason
print(pbe0['Unnamed: 0'])
print(lda['Unnamed: 0'])

0      bicyclo[3.1.0]hexane
1                 phosphine
2         beryllium_sulfide
3             lithium_oxide
4           carbon_monoxide
               ...         
220            cyclopropene
221             cyclobutane
222              silylene_s
223    aluminum_monohydride
224         beryllium_oxide
Name: Unnamed: 0, Length: 225, dtype: object
0      bicyclo[3.1.0]hexane
1                 phosphine
2         beryllium_sulfide
3             lithium_oxide
4           carbon_monoxide
               ...         
220            cyclopropene
221             cyclobutane
222              silylene_s
223    aluminum_monohydride
224         beryllium_oxide
Name: Unnamed: 0, Length: 225, dtype: object


In [4]:
print(pbe0['Energy'])

0     -234.448220
1     -343.028857
2     -412.752433
3      -82.619718
4     -113.237996
          ...    
220   -116.526064
221   -157.084428
222   -290.496811
223   -242.867793
224    -89.829275
Name: Energy, Length: 225, dtype: float64


In [5]:
# the whole csv file (dipoles, forces, wall time etc)
print(pbe0)

               Unnamed: 0      Energy  Iterations  Dipole Moment  \
0    bicyclo[3.1.0]hexane -234.448220          12   7.575750e-02   
1               phosphine -343.028857          10   2.507087e-01   
2       beryllium_sulfide -412.752433          14   2.164581e+00   
3           lithium_oxide  -82.619718          11   2.568827e+00   
4         carbon_monoxide -113.237996           9   3.864714e-02   
..                    ...         ...         ...            ...   
220          cyclopropene -116.526064          10   1.839676e-01   
221           cyclobutane -157.084428          10   7.186659e-07   
222            silylene_s -290.496811          10   8.775786e-02   
223  aluminum_monohydride -242.867793          10   2.059471e-02   
224       beryllium_oxide  -89.829275          13   2.573871e+00   

                                                Eigens  \
0    [-10.22198465, -10.22168891, -10.21485057, -10...   
1    [-77.20463688, -6.60599131, -4.74144868, -4.74...   
2    [-88

In [6]:
# go through input files in geometry folder and parse geometry block into dictonary mols
#
# get relative path assuming geometries are in separate folder in the same directory
path = os.path.abspath(os.getcwd())
geometries_path = os.path.join(path, 'geometries')
mols = {}
# if relative path doesn't work use absolute path instead
#for filename in os.scandir("/home/hannes/Documents/SBU/AMS561/ML/geometries"):
for filename in os.scandir(geometries_path):
    # parse name of molecule (filename) for key
    # does not distinguish files - make sure ONLY mdns files are in director
    # e.g. no swap files from editors
    name = str(filename).split()[1]
    name = name.replace('.mdns', '')
    name = name.replace("'", "")
    name = name.replace(">", "")
    if filename.is_file():
        f = open(filename, "r")
        # list of lists containing [letter, x, y, z]
        mol = []
        # the format is
        # geometry
        # units angstrom
        #  H 0 0 0
        #  ...
        # end
        while True:
            line = f.readline()
            # skip the angstrom and geometry lines (not very clean but works for now)
            if "geometry" in line:
                line = f.readline()
                #if "units angstrom" in line:
                 #   f.readline()
                while True:
                    line = f.readline()
                    if line == "end":
                        break
                    mol.append(line.split())
                mols[name] = mol        
            if not line:
                break

In [7]:
print(len(mols))

226


In [8]:
print(mols["water"])

[['O', '0.000000000000', '0.000000000000', '0.117300000000'], ['H', '0.000000000000', '0.757200000000', '-0.469200000000'], ['H', '0.000000000000', '-0.757200000000', '-0.469200000000']]


In [9]:
# print a molecule
mol_name = "phosphine"
for i in mols[mol_name]:
    print(i)

['P', '0.000000000000', '0.000000000000', '0.000000000000']
['H', '0.000000000000', '-1.193200000000', '-0.771700000000']
['H', '1.033300000000', '0.596600000000', '-0.771700000000']
['H', '-1.033300000000', '0.596600000000', '-0.771700000000']
